In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [49]:
from pathlib import Path
def load_asm_file(file_path):
    def read_lines():
        file_content = Path(file_path).read_text().replace('\r', '')
        lines = file_content.split('\n')
        return lines

    def remove_empty_lines(lines):
        non_empty_lines = [l.strip() for l in lines if l.strip() and not l.strip().startswith('//')]
        return non_empty_lines

    lines = read_lines()
    non_empty_lines = remove_empty_lines(lines)
    return non_empty_lines

In [42]:
def symbols(lines):
    def is_label(line):
        return line.startswith('(') and line.endswith(')')

    def extract_label_name(label_declaration):
        return label_declaration.strip('()')

    def decimal_binary(lookuptable):
        binary_dict = {}
        for key, value in lookuptable.items():
            binary_representation = bin(value)[2:]
            binary_representation = binary_representation.rjust(16, '0')
            binary_dict[key] = binary_representation
        return binary_dict

    def variablesymbols(lines, lookuptable):
        variablestart = 16
        for i in lines:
            if i.startswith("@"):
                variable_name = i[1:]
                if variable_name not in lookuptable:
                    if variable_name.isdigit():
                        lookuptable[variable_name] = int(variable_name)
                    else:
                        lookuptable[variable_name] = variablestart
                        variablestart += 1
        return lookuptable

    # Main function logic
    lookuptable = {
        "SCREEN": 16384,
        "KBD": 24576,
        "SP": 0,
        "LCL": 1,
        "ARG": 2,
        "THIS": 3,
        "THAT": 4
    }

    for i in range(16):
        label = "R" + str(i)
        lookuptable[label] = i

    program_counter = 0

    for line in lines:
        if is_label(line):
            label_name = extract_label_name(line)
            lookuptable[label_name] = program_counter
        else:
            program_counter += 1

    nlookuptable = variablesymbols(lines, lookuptable)

    return decimal_binary(nlookuptable)


In [46]:
def c_instruction(lines):
    comp = {
        "0": "0101010",
        "1": "0111111",
        "-1": "0111010",
        "D": "0001100",
        "A": "0110000",
        "!D": "0001101",
        "!A": "0110001",
        "-D": "0001111",
        "-A": "0110011",
        "D+1": "0011111",
        "A+1": "0110111",
        "D-1": "0001110",
        "A-1": "0110010",
        "D+A": "0000010",
        "D-A": "0010011",
        "A-D": "0000111",
        "D&A": "0000000",
        "D|A": "0010101",
        "M": "1110000",
        "!M": "1110001",
        "-M": "1110011",
        "M+1": "1110111",
        "M-1": "1110010",
        "D+M": "1000010",
        "D-M": "1010011",
        "M-D": "1000111",
        "D&M": "1000000",
        "D|M": "1010101"
    }

    dest = {
        "null": "000",
        "M": "001",
        "D": "010",
        "A": "100",
        "MD": "011",
        "AM": "101",
        "AD": "110",
        "AMD": "111"
    }

    jump = {
        "null": "000",
        "JGT": "001",
        "JEQ": "010",
        "JGE": "011",
        "JLT": "100",
        "JNE": "101",
        "JLE": "110",
        "JMP": "111"
    }

    binary_instructions = []

    for line in lines:
        if line.startswith('@') or (line.startswith('(') and line.endswith(')')):
            # Skip A-instructions and labels
            continue

        # Normalize C-instruction format
        if "=" not in line:
            line = "null=" + line
        if ";" not in line:
            line = line + ";null"

        temp = line.split("=")
        destCode = dest.get(temp[0])
        temp = temp[1].split(";")
        compCode = comp.get(temp[0])
        jumpCode = jump.get(temp[1])

        binary_instructions.append("111" + compCode + destCode + jumpCode)

    return binary_instructions




In [48]:
file_path = '/content/drive/MyDrive/Max.asm'
lines = load_asm_file(file_path)
binary_dict = symbols(lines)

for line in lines:
    if line.startswith("@") :
        print(binary_dict.get(line[1:]))
    elif line.startswith('(') and line.endswith(')'):
        print(binary_dict.get(line.strip("( )")))
    else:
        binary_instructions = c_instruction([line])
        print(binary_instructions[0])



0000000000000000
1111110000010000
0000000000000001
1111010011010000
0000000000001100
1110001100000001
0000000000000001
1111110000010000
0000000000000010
1110001100001000
0000000000010000
1110101010000111
0000000000001100
0000000000000000
1111110000010000
0000000000000010
1110001100001000
0000000000010000
0000000000010000
1110101010000111


In [ ]:
# add.asm file

#// This file is part of www.nand2tetris.org
# // and the book "The Elements of Computing Systems"
# // by Nisan and Schocken, MIT Press.
# // File name: projects/06/add/Add.asm

# // Computes R0 = 2 + 3  (R0 refers to RAM[0])

# //@2
# //D=A
# //@3
# //D=D+A
# //@0
# //M=D